# **Chapter 7: Synchronization Examples – Practice Exercise Solutions**

## **7.1 Multiple Locking Mechanisms in Windows and Linux**

**Question:**  
Explain why Windows and Linux implement multiple locking mechanisms. Describe the circumstances under which they use spinlocks, mutex locks, semaphores, and condition variables. In each case, explain why the mechanism is needed.

**Answer:**  
These operating systems provide different locking mechanisms depending on the application developers’ needs.  
- **Spinlocks**: Used in **multiprocessor systems** for **short-duration** critical sections where the cost of context switching (sleep/wake) is higher than briefly spinning. They avoid scheduler overhead.  
- **Mutex locks**: Used for general **mutual exclusion** when a resource may be held for longer periods. They block the thread (put it to sleep) if the lock is unavailable, saving CPU cycles. Some systems (like Solaris) use **adaptive mutexes** that spin briefly before blocking.  
- **Semaphores**: Used for **counting resources** (e.g., limiting concurrent accesses) or synchronizing between threads/processes where a thread may need to wait for a signal from another. Useful for **long waits**.  
- **Condition variables**: Used with mutexes for **complex synchronization patterns** (e.g., producer-consumer) where a thread must wait for a specific condition to become true. They allow efficient waiting without busy polling.

Each mechanism addresses different performance and correctness trade-offs.

---

## **7.2 Slim Reader-Writer Locks in Windows**

**Question:**  
Windows provides a lightweight synchronization tool called slim reader–writer locks. Whereas most implementations of reader–writer locks favor either readers or writers, or perhaps order waiting threads using a FIFO policy, slim reader–writer locks favor neither readers nor writers, nor are waiting threads ordered in a FIFO queue. Explain the benefits of providing such a synchronization tool.

**Answer:**  
**Simplicity and speed**. Slim reader-writer locks avoid the overhead of fairness policies (like FIFO ordering) or favoring readers/writers, which require additional queue management and state tracking. This makes acquisition and release **faster** with lower memory overhead. They are appropriate when strict fairness is not required, but fast, lightweight read-write synchronization is needed—common in high-performance kernel or user-space code where lock contention is moderate.

---

## **7.3 Replacing Binary Semaphore with Mutex Lock**

**Question:**  
Describe what changes would be necessary to the producer and consumer processes in Figure 7.1 and Figure 7.2 so that a mutex lock could be used instead of a binary semaphore.

**Answer:**  
The calls to `wait(mutex)` and `signal(mutex)` (where `mutex` is a binary semaphore) need to be replaced with the **mutex lock API** calls:  
- `wait(mutex)` → `acquire(mutex)` or `lock(mutex)`  
- `signal(mutex)` → `release(mutex)` or `unlock(mutex)`  

The logic remains identical; only the synchronization primitive changes. The mutex lock provides the same mutual exclusion but is often implemented with support for ownership (to prevent a thread from unlocking a mutex it doesn’t hold) and may include priority inheritance features.

---

## **7.4 Deadlock in Dining Philosophers Problem**

**Question:**  
Describe how deadlock is possible with the dining-philosophers problem.

**Answer:**  
If all philosophers simultaneously pick up their **left fork**, each will hold one fork and wait indefinitely for their **right fork**, which is held by the neighbor to their right. Since no philosopher puts down a fork until they eat, all are blocked forever—a **circular wait** condition leading to deadlock.

---

## **7.5 Signaled vs. Non-Signaled States in Windows**

**Question:**  
Explain the difference between signaled and non-signaled states with Windows dispatcher objects.

**Answer:**  
- **Signaled state**: The object is **available**. A thread attempting to acquire it (e.g., via `WaitForSingleObject`) will **not block** and will proceed immediately. The object may represent a lock that is free, an event that has occurred, etc.  
- **Non-signaled state**: The object is **unavailable**. A thread trying to acquire it will **block** until the object transitions to signaled (e.g., when a lock is released or an event is set).

---

## **7.6 Atomic Operations in Linux**

**Question:**  
Assume `val` is an atomic integer in a Linux system. What is the value of `val` after the following operations have been completed?  
```
atomic_set(&val,10);
atomic_sub(8,&val);
atomic_inc(&val);
atomic_inc(&val);
atomic_add(6,&val);
atomic_sub(3,&val);
```

**Answer:**  
Calculating sequentially:  
1. `atomic_set(&val,10)` → val = 10  
2. `atomic_sub(8,&val)` → val = 10 - 8 = 2  
3. `atomic_inc(&val)` → val = 2 + 1 = 3  
4. `atomic_inc(&val)` → val = 3 + 1 = 4  
5. `atomic_add(6,&val)` → val = 4 + 6 = 10  
6. `atomic_sub(3,&val)` → val = 10 - 3 = 7  

**Final value of val is 7**.


# **Chapter 7: Synchronization Examples – Exercise Solutions**

## **Exercise 7.12**
**Question:**  
Describe two kernel data structures in which race conditions are possible. Be sure to include a description of how a race condition can occur.

**Answer:**  
1. **Process/Thread Control Block (PCB/TCB) Linked List**  
   - **Race Condition**: When a new process is created or terminated, the kernel must update the linked list of PCBs. If two CPUs simultaneously attempt to insert or remove a PCB (e.g., during `fork()` and `exit()`), they may both manipulate the `next` pointers concurrently, corrupting the list. For example, both might read the same head pointer, then update it to point to their own new PCB, causing one insertion to be lost or creating a loop.

2. **Memory Allocation Buddy System Free List**  
   - **Race Condition**: The buddy system maintains lists of free memory blocks of different sizes. When allocating or freeing a block, the kernel must remove or insert blocks from these lists. If two processes on different cores simultaneously request memory of the same size, they might both find the same free block in the list, both attempt to remove it, and both believe they have acquired it, leading to double allocation and potential memory corruption.

---

## **Exercise 7.13**
**Question:**  
The Linux kernel has a policy that a process cannot hold a spinlock while attempting to acquire a semaphore. Explain why this policy is in place.

**Answer:**  
This policy prevents **deadlock** and **priority inversion** scenarios.  
- A spinlock is designed for **short-term** locking in contexts where sleeping is not allowed (e.g., interrupt handlers). Holding a spinlock and then trying to acquire a semaphore (which may **block/sleep** if unavailable) would cause the thread to sleep while holding the spinlock.  
- If the thread sleeps, another thread on the same CPU might try to acquire the same spinlock, causing **deadlock** because the lock is held by a sleeping thread.  
- Additionally, spinning with a held semaphore could waste CPU indefinitely if the semaphore is held by a low-priority process that gets preempted.  
The policy enforces that spinlocks are only used in non-blocking contexts.

---

## **Exercise 7.14**
**Question:**  
Design an algorithm for a bounded-buffer monitor in which the buffers (portions) are embedded within the monitor itself.

**Answer:**  
Monitor with internal buffer array and condition variables for empty/full.

```java
monitor BoundedBuffer {
    private int buffer[N];
    private int count = 0, in = 0, out = 0;
    condition notFull, notEmpty;

    void produce(int item) {
        while (count == N) {
            notFull.wait();  // wait until buffer not full
        }
        buffer[in] = item;
        in = (in + 1) % N;
        count++;
        notEmpty.signal();  // signal consumer
    }

    int consume() {
        while (count == 0) {
            notEmpty.wait(); // wait until buffer not empty
        }
        int item = buffer[out];
        out = (out + 1) % N;
        count--;
        notFull.signal();   // signal producer
        return item;
    }
}
```

---

## **Exercise 7.15**
**Question:**  
The strict mutual exclusion within a monitor makes the bounded-buffer monitor of Exercise 7.14 mainly suitable for small portions.
a. Explain why this is true.
b. Design a new scheme that is suitable for larger portions.

**Answer:**  
**a.** Strict mutual exclusion means only one thread can be active inside the monitor at a time. If the buffer operations (`produce`/`consume`) involve **large data portions** (e.g., copying large chunks of memory), the critical section becomes long. This serializes all producers and consumers, hurting **throughput** and **parallelism** unnecessarily, since multiple producers could fill different buffer slots concurrently, and multiple consumers could take from different slots concurrently.

**b.** **Scheme for larger portions**: Use a **reader-writer** style approach or **fine-grained locking**.
- Split the buffer into **multiple slots**, each with its own lock (or use an array of semaphores).
- Producers and consumers can operate on **different slots simultaneously**.
- Maintain shared counters (`count`, `in`, `out`) with atomic operations or a separate lock.

**Pseudocode using multiple semaphores:**
```c
semaphore mutex = 1;          // protects in/out/count
semaphore empty = N;          // counts empty slots
semaphore full = 0;           // counts full slots
semaphore slot_locks[N] = {1,1,...}; // one per slot

void produce(int item, int slot_index) {
    wait(empty);              // ensure space
    wait(mutex);
    // find free slot (could be predefined), e.g., slot_index = in;
    wait(slot_locks[slot_index]); // lock this slot only
    buffer[slot_index] = item; // copy large data
    update in, count;
    signal(mutex);
    signal(slot_locks[slot_index]);
    signal(full);
}

int consume(int slot_index) {
    wait(full);
    wait(mutex);
    // find full slot, e.g., slot_index = out;
    wait(slot_locks[slot_index]);
    int item = buffer[slot_index];
    update out, count;
    signal(mutex);
    signal(slot_locks[slot_index]);
    signal(empty);
    return item;
}
```
This allows concurrent data copying into different buffer slots.

---

## **Exercise 7.16**
**Question:**  
Discuss the tradeoff between fairness and throughput of operations in the readers–writers problem. Propose a method for solving the readers–writers problem without causing starvation.

**Answer:**  
**Tradeoff**:  
- **Throughput favor (Reader priority)**: Allows multiple readers simultaneously, maximizing read throughput, but **writers may starve** if readers keep arriving.  
- **Fairness (Writer priority or FIFO)**: Writers get timely access, preventing starvation, but may reduce read throughput because readers are serialized or forced to wait.  

**Starvation-Free Solution**:  
Use a **fair scheduling policy** like:
1. **Timestamp ordering**: Arriving threads (readers/writers) get a timestamp. Service in order; a writer blocks all later arrivals; a reader allows other readers with earlier timestamps to join.
2. **The "No Starvation" Readers-Writers Lock**:  
   - Use two mutexes: `resource_mutex` (for writers) and `read_count_mutex` (for reader count).  
   - Add a **turnstile** semaphore or a **queue semaphore** to enforce FIFO order for both readers and writers.  
   - When a writer is waiting, new readers are blocked until the writer finishes.  
   - Implementation often uses a **fair semaphore** (like `pthread_rwlock` with priority to writers) or **condition variables with a queue**.

**Example using condition variables:**
```java
monitor FairRW {
    int readers = 0;
    bool writing = false;
    int waitingWriters = 0;
    condition okToRead, okToWrite;

    void startRead() {
        while (writing || waitingWriters > 0) {
            okToRead.wait();
        }
        readers++;
        okToRead.signal(); // cascade to other waiting readers
    }

    void endRead() {
        readers--;
        if (readers == 0) {
            okToWrite.signal();
        }
    }

    void startWrite() {
        waitingWriters++;
        while (readers > 0 || writing) {
            okToWrite.wait();
        }
        waitingWriters--;
        writing = true;
    }

    void endWrite() {
        writing = false;
        if (waitingWriters > 0) {
            okToWrite.signal();
        } else {
            okToRead.signal();
        }
    }
}
```
This gives writers priority but ensures readers eventually proceed.

---

## **Exercise 7.17**
**Question:**  
Explain why the call to the `lock()` method in a Java `ReentrantLock` is not placed in the `try` clause for exception handling, yet the call to the `unlock()` method is placed in a `finally` clause.

**Answer:**  
- `lock()` is not placed in `try` because if `lock()` throws an exception (e.g., `InterruptedException`), the lock has **not been acquired**, so we should **not** attempt to unlock it in `finally`. Unlocking without ownership would cause an `IllegalMonitorStateException`.  
- `unlock()` is placed in `finally` to **guarantee** that the lock is released if it was successfully acquired, even if an exception occurs in the critical section. This prevents deadlock.

**Typical pattern:**
```java
ReentrantLock lock = new ReentrantLock();
lock.lock();  // outside try
try {
    // critical section
} finally {
    lock.unlock(); // always unlock if locked
}
```

---

## **Exercise 7.18**
**Question:**  
Explain the difference between software and hardware transactional memory.

**Answer:**  
- **Software Transactional Memory (STM)**: Implemented entirely in software (libraries, compiler support). It uses **logging**, **versioning**, or **copy-on-write** to group memory operations into atomic transactions. Conflicts are detected via software algorithms (e.g., using locks or timestamps). Higher overhead but portable across hardware.

- **Hardware Transactional Memory (HTM)**: Supported directly by CPU hardware (e.g., Intel TSX, IBM POWER). Uses **cache coherence protocols** to detect conflicts and CPU caches to buffer speculative writes. Transactions are executed speculatively; if a conflict occurs (another core accesses same memory), hardware aborts and rolls back. Lower overhead, faster for small transactions, but limited by hardware constraints (cache size, instruction support).

**Key difference**: HTM is faster and transparent to programmer but limited in transaction size and not universally available. STM is more flexible (larger transactions) but slower due to software overhead.

--- 

**End of Chapter 7 Exercises**